In [ ]:
import tensorflow as tf
import os
_url="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data=tf.keras.utils.get_file(origin=_url,fname="flower_photos.tgz",extract=True,cache_subdir=('/content'))
base_dir=os.path.join(os.path.dirname(data),'flower_photos')

228818944/228813984 [==============================] - 2s 0us/step


In [ ]:
IMG_SIZE=224
Batch_Size=32


datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

train_generator=datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_SIZE,IMG_SIZE),batch_size=Batch_Size,
    subset="training"
)

val_generator=datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_SIZE,IMG_SIZE),batch_size=Batch_Size,
    subset="validation"
)


Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [ ]:
labels='\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt','w') as f:
  f.write(labels)


In [ ]:
IMG_SHAPE=(IMG_SIZE,IMG_SIZE,3)
base_model=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable=False
model=tf.keras.Sequential([
            base_model,
            tf.keras.layers.Conv2D(32,3,activation="relu"),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(5,activation="softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 2,626,821
Trainable params: 368,837
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,
                  validation_data=val_generator,
                  epochs=10)

Epoch 1/10
92/92 [==============================] - 17s 186ms/step - loss: 0.7377 - accuracy: 0.7509 - val_loss: 0.4595 - val_accuracy: 0.8468
Epoch 2/10
92/92 [==============================] - 16s 170ms/step - loss: 0.3043 - accuracy: 0.8945 - val_loss: 0.4219 - val_accuracy: 0.8659
Epoch 3/10
92/92 [==============================] - 16s 169ms/step - loss: 0.1796 - accuracy: 0.9388 - val_loss: 0.3880 - val_accuracy: 0.8700
Epoch 4/10
92/92 [==============================] - 16s 170ms/step - loss: 0.1130 - accuracy: 0.9626 - val_loss: 0.4729 - val_accuracy: 0.8659
Epoch 5/10
92/92 [==============================] - 16s 169ms/step - loss: 0.0811 - accuracy: 0.9755 - val_loss: 0.4397 - val_accuracy: 0.8687
Epoch 6/10
92/92 [==============================] - 16s 169ms/step - loss: 0.0591 - accuracy: 0.9823 - val_loss: 0.4718 - val_accuracy: 0.8591
Epoch 7/10
92/92 [==============================] - 16s 169ms/step - loss: 0.0437 - accuracy: 0.9888 - val_loss: 0.6447 - val_accuracy: 0.8413

In [ ]:
saved_model_dir=""
tf.saved_model.save(model,saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
tflite_model = converter.convert()


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: assets


In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>